In [1]:
# Импортируем необходимые модули
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from datetime import date
import os

Теперь необходимо собрать информацию со страниц, ссылки на которые мы получили из карточек объявлений на предыдущем этапе.      
        
*Примечание: сбор данных проходил на ОС Windows 10* 

In [ ]:
# Считываем первый файл с ссылками, полученный на предыдущем этапе
data = pd.read_csv('./data/auto_vol/auto_0.csv')

# В цикле добавляем ссылки из остальных файлов
for i in range(1, 100):
    df = pd.read_csv(f'./data/auto_vol/auto_{i}.csv')
    data = pd.concat([data, df])

# Проверяем результат
display(data.shape)

(3757, 2)

In [ ]:
# Выделим ссылки в отдельный столбец
data['url'] = data['link'].apply(lambda x: x.split('"')[9])

# Удалим дубоикаты и проверим результат
data.drop_duplicates(inplace=True)
display(data.shape)

(2138, 3)

In [ ]:
# Получаем список ссылок на страницы
urls = data['url'].to_list()

# В цикле перебираем ссылки. Иногда парсер может дать сбой поэтому 
# на каждой итерации будем дозаписывать собранную информацию в файл. 
# В случае сбоя можно будет продолжить сбор с соответствующей ссылки
for num, url in enumerate(urls[992:]):
    
    # Печатаем номер ссылки для контроля процесса
    print(num)
    
    # Записываем текущую дату 
    parse_date = date.today().strftime('%d-%m-%Y')
    
    # запускаем вебдрайвер и передаем в него ссылку на страницу
    driver = webdriver.Firefox()
    driver.get(url=url)
    
    # включаем окно во весь экран
    driver.maximize_window()
    
    # Объявление может быть уже удалено, поэтому пробуем найти заголовок
    try:
        # Собираем информацию 
        head = driver.find_element(By.CLASS_NAME, "CardHead__topRow")
        heads_all = [head.text]
        
        # Машина может быть уже продана тогда ячейку с продавцом и ценой мы не найдем
        try:
            owner = driver.find_element(By.CLASS_NAME, "CardOwner__firstCell")
            owners_all = [owner.text]                  
        except:
            try:
                # вместо неё будет ячейка с надписью, что машина уже продана, класс у этой ячейки уже другой
                owner = driver.find_element(By.CLASS_NAME, "CardSold")
                owners_all = [owner.text]
            except:
                try:
                    # Иногда вместо продавца может быть указано, что машина продается через автовыкуп
                    owner = driver.find_element(By.CLASS_NAME, "C2BAuctionStatusBannerSellingBuyer__info-W5mzV")
                    # Записываем кто продает
                    owner_beneficts = driver.find_element(By.CLASS_NAME, "CardBenefits__item-description")
                    owners_all = [owner.text + "\n" + owner_beneficts.text]
                except:                    
                    # Если оценка еще проводится, то имени продавца не будет. Записываем сообщение об оценке
                    owner_scoring = driver.find_element(By.CLASS_NAME, "C2BAuctionStatusBannerSellingBuyer__subtitle-xSbfw")
                    owners_all = [owner.text + "\n" + owner_scoring.text]
           
        
        # Находим и записываем характеристики машины    
        characters = driver.find_element(By.CLASS_NAME, "CardInfo-ateuv")
        characters_all = [characters.text]
        
        # Собираем информацию из поля комменатрия от продовца
        try:
            # Поле с комментарием продавца может отсутствовать, поэтому пробуем найти его
            # Перемещаемся к полю комментария
            anchor_page_1 = driver.find_element(By.CSS_SELECTOR, '.CardDescription__titleWrap')
            driver.execute_script("arguments[0].scrollIntoView(true);", anchor_page_1)
            
            # Требуется нажатие кнопки, чтобы вывести весь комментарий
            button_comment = driver.find_element(By.CSS_SELECTOR, '.CardDescription__cutLink')
            
            # Ждем когда кнопка станет кликабельной
            comm_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button_comment))
            
            # Жмем кнопку       
            ActionChains(driver).move_to_element(comm_button).click().perform()
                      
            # Находим комментарий и записываем его
            comment = driver.find_element(By.CLASS_NAME, "CardDescriptionHTML")
            comments_all = [comment.text]
            
        
        except:
            try:
                # Если комментарий короткий - кнопки для его показа не будет, поэтому просто находим 
                # и записываем его 
                comment = driver.find_element(By.CLASS_NAME, "CardDescriptionHTML")
                comments_all = [comment.text]
                
            # Если комментария нет записываем 'None'
            except:
                comments_all = ['None']
        
        # Собираем информацию из поля опции        
        try:
            # Поля с опциями тоже может не быть, пробуем его найти 
            # перемещаемся к полю комплектации
            anchor_page_2 = driver.find_element(By.CSS_SELECTOR, '.CardComplectation__title')
            driver.execute_script("arguments[0].scrollIntoView(true);", anchor_page_2)            
            
            # Требуется нажатие кнопки, чтобы вывести всю комплектацию
            button = driver.find_element(By.CSS_SELECTOR, '.ComplectationGroupsDesktop__cutLink')
            
            # Ждем когда кнопка станет кликабельной
            options_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button))
            
            # Жмем кнопку                                                 
            ActionChains(driver).move_to_element(options_button).click().perform()
            
            # Опций может быть очень много, поэтому прокручиваем окно вниз на 600 пикселей (этого 
            # по опыту хватает), находим поле с опциями и записываем информацию из него
            driver. execute_script("window. scrollBy(0, 600)")
            options = driver.find_element(By.CLASS_NAME, "ComplectationGroupsDesktop__row")
            options_all = [options.text]
            
        except:
            # Если опций мало - кнопки не будет, поэтому просто ищем объект и записываем информацию из него 
            try:
                options = driver.find_element(By.CLASS_NAME, "ComplectationGroupsDesktop__row")
                options_all = [options.text]
                
            # Если опций нет записываем 'None'    
            except:
                options_all = ['None']
    
    except:
        # Если объявление удалено заголовка мы не найдем, поэтому записываем во все переменные 'None'
        heads_all = ['None']
        owners_all = ['None']
        characters_all = ['None']
        comments_all = ['None']
        options_all = ['None']
    
    # Создаем словарь из полученных списков
    frame_data = {'head':heads_all,
                  'owner':owners_all,
                  'characters':characters_all,
                  'comment':comments_all,
                  'options':options_all,
                  'parse_date':parse_date
                  }
    
    # Преобразуем словарь в датафрейм
    df = pd.DataFrame(frame_data)
    
    # Дозаписываем датафрейм в файл
    df.to_csv('./data/auto_vol/auto_vol_info_0+.csv', mode='a', index=False, header=False)
    
    # Выводим сообщение для контроля процесса
    print('save done')
    
    # Закрываем вебдрайвер (окно браузера)
    driver.close()
    
# Выходим из вебдрайвера
driver.quit()    

0
save done
1
save done
2
save done
3
save done
4
save done
5
save done
6
save done
7
save done
8
save done
9
save done
10
save done
11
save done
12
save done
13
save done
14
save done
15
save done
16
save done
17
save done
18
save done
19
save done
20
save done
21
save done
22
save done
23
save done
24
save done
25
save done
26
save done
27
save done
28
save done
29
save done
30
save done
31
save done
32
save done
33
save done
34
save done
35
save done
36
save done
37
save done
38
save done
39
save done
40
save done
41
save done
42
save done
43
save done
44
save done
45
save done
46
save done
47
save done
48
save done
49
save done
50
save done
51
save done
52
save done
53
save done
54
save done
55
save done
56
save done
57
save done
58
save done
59
save done
60
save done
61
save done
62
save done
63
save done
64
save done
65
save done
66
save done
67
save done
68
save done
69
save done
70
save done
71
save done
72
save done
73
save done
74
save done
75
save done
76
save done
77
save d

In [ ]:
# Загружаем данные, присваиваем имена столбцам и проверяем результат
df_auto = pd.read_csv('./data/auto_vol/auto_vol_info_0+.csv', header=None, names=['head',
                                                                                  'owner',
                                                                                  'characters',
                                                                                  'comment',
                                                                                  'options',
                                                                                  'parse_date'
                                                                                  ]
                      )
display(df_auto)

,head,owner,characters,comment,options,parse_date
0,"Mitsubishi Outlander, 2016\n20 сентября\n19 (1...","Автобро\nВологдаулица Ильюшина, 28А · 176 авто...",Наличие\nВ наличии\nПоколение\nIII Рестайлинг ...,-Два владельца\n-Оригинальный ПТС\n-За весь пе...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
1,"Volkswagen Amarok Double Cab full-time, 2012\n...",Динамика Череповец\nЧереповецОктябрьский просп...,Наличие\nВ наличии\nПоколение\nI\nГод выпуска\...,👀 ВНИМАНИЕ!\nСЕНСАЦИОННАЯ ПРИВИЛЕГИЯ МЕСЯЦА!\n...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
2,"Lada (ВАЗ) Priora, 2011\n30 августа\n528 (65 с...",Динамика Череповец\nЧереповецОктябрьский просп...,Наличие\nВ наличии\nПоколение\nI\nГод выпуска\...,👀 ВНИМАНИЕ!\nСЕНСАЦИОННАЯ ПРИВИЛЕГИЯ МЕСЯЦА!\n...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
3,"Mitsubishi L200, 2014\n18 сентября\n248 (76 се...","Автобро\nВологдаулица Ильюшина, 28А · 176 авто...",Наличие\nВ наличии\nПоколение\nIV Рестайлинг\n...,- ПТС оригинал\n- Богатая комплектация\n- Полн...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
4,"Mitsubishi L200, 2011\n20 сентября\n47 (47 сег...",Мартен | Автомобили с пробегом\nВологдаОкружно...,Наличие\nВ наличии\nПоколение\nIV\nГод выпуска...,Mitsubishi L200 2.5d МТ\n- 2 собственника\n-Зе...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
...,...,...,...,...,...,...
2133,"Lada (ВАЗ) Vesta, 2023\n30 августа\n88 (1 сего...","АВТОМАРКЕТ\nЯрославльулица Полушкина Роща, 11 ...",Наличие\nВ наличии\nПоколение\nI Рестайлинг (N...,• Начало эксплуатации 18.08.2023г.\n• Автомоби...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
2134,"Opel Astra, 2010\n14 ноября 2023\n544\n№ 11212...",Жук - жук\nУглич,Наличие\nВ наличии\nПоколение\nJ\nГод выпуска\...,Продается Opel Astra 2010 года выпуска. Автомо...,Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
2135,"Kia Rio 4-speed, 2012\n10 августа\n120 (1 сего...","Частное лицо\nЧереповецпроспект Победы, 115",Наличие\nВ наличии\nПоколение\nIII\nГод выпуск...,"Оличное состояние, все работает, масло не ест,...",Безопасность\n•\nПодушка безопасности водителя...,21-09-2024
2136,"Toyota Land Cruiser Prado, 2016\n23 июня\n182\...","Частное лицо\nКирилловулица Урицкого, 3",Наличие\nВ наличии\nПоколение\n150 Series Рест...,Продаю свой надежный прадик. На рынке очень ма...,Салон\n•\nКоличество мест: 5,21-09-2024


In [ ]:
# Записываем датафрейм в файл
df_auto.to_csv('./data/fin/vol_data.csv', index=False)